<a href="https://colab.research.google.com/github/aiqn1988/bulk/blob/main/Clone_ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [ ]:
#@title Check GPU
!nvidia-smi

Sat Nov 25 07:46:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Install dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
!python -m pip install -U pip wheel
%pip install -U ipython

#@markdown Branch (for development)
BRANCH = "none" #@param {"type": "string"}
if BRANCH == "none":
    %pip install -U so-vits-svc-fork
else:
    %pip install -U git+https://github.com/34j/so-vits-svc-fork.git@{BRANCH}

## Training

In [21]:
#@title Make dataset directory
!mkdir -p "dataset_raw"

In [22]:
#!rm -r "dataset_raw"
#!rm -r "dataset/44k"

In [23]:
#@title Copy your dataset
#@markdown **We assume that your dataset is in your Google Drive's `so-vits-svc-fork/dataset/(speaker_name)` directory.**
DATASET_NAME = "vutrunguyenthuy" #@param {type: "string"}
!cp -R /content/drive/MyDrive/aimodel/{DATASET_NAME}/ -t "dataset_raw/"

In [ ]:
#@title Download dataset (Tsukuyomi-chan JVS)
#@markdown You can download this dataset if you don't have your own dataset.
#@markdown Make sure you agree to the license when using this dataset.
#@markdown https://tyc.rei-yumesaki.net/material/corpus/#toc6
# !wget https://tyc.rei-yumesaki.net/files/sozai-tyc-corpus1.zip
# !unzip sozai-tyc-corpus1.zip
# !mv "/content/つくよみちゃんコーパス Vol.1 声優統計コーパス（JVSコーパス準拠）/おまけ：WAV（+12dB増幅＆高音域削減）/WAV（+12dB増幅＆高音域削減）" "dataset_raw/tsukuyomi"

In [24]:
#@title Automatic preprocessing
!svc pre-resample

Preprocessing: 100% 311/311 [00:18<00:00, 17.17it/s]


In [25]:
!svc pre-config

  0% 0/311 [00:00<?, ?it/s][06:15:18] WARNING  [06:15:18]                                                       ]8;id=187526;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=23027;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                    /usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/preproc                
                    essing/preprocess_flist_config.py:41: FutureWarning:                            
                    get_duration() keyword argument 'filename' has been renamed to                  
                    'path' in version 0.10.0.                                                       
                            This alias will be removed in version 1.0.                              
                      if get_duration(filename=path) < 0.3:                                         
                                                                                                    
100% 311/311 [00:00<00:00, 1587.47it/s]


In [28]:
#@title Copy configs file
!cp configs/44k/config.json drive/MyDrive/aimodel

In [29]:
F0_METHOD = "dio" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
!svc pre-hubert -fm {F0_METHOD}

[06:22:54] INFO     [06:22:54] n_jobs automatically set to 2, memory:    ]8;id=950876;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/preprocessing/preprocess_hubert_f0.py\preprocess_hubert_f0.py]8;;\:]8;id=487645;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/preprocessing/preprocess_hubert_f0.py#142\142]8;;\
                    15360 MiB                                                                       
Some weights of HubertModel were not initialized from the model checkpoint at lengyue233/content-vec-best and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of HubertModel were not initialized from the model checkpoint at lengyue233/content-vec-best and are newly initialized: ['encoder.pos_conv_embed.conv.parametr

In [ ]:
#@title Train
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/aimodel/logs/44k
!svc train --model-path drive/MyDrive/aimodel/logs/44k
%reload_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-11-26 06:25:51.586106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 06:25:51.586177: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 06:25:51.586209: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 06:25:51.604925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appro

Streaming output truncated to the last 5000 lines.
Epoch 193/299 ━━━━━━━━━━━╺━━━━━━━━━━━━━━ 17/39 0:00:22 • 0:00:29 0.77it/s v_num: 0.000 loss/g/total:
                                                                          30.446 loss/g/fm: 9.782   
                                                                          loss/g/mel: 16.823        
                                                                          loss/g/kl: 0.837          
                                                                          loss/g/lf0: 0.000         
Epoch 193/299 ━━━━━━━━━━━━╺━━━━━━━━━━━━━ 18/39 0:00:23 • 0:00:28 0.77it/s v_num: 0.000 loss/g/total:
                                                                          30.446 loss/g/fm: 9.782   
                                                                          loss/g/mel: 16.823        
                                                                          loss/g/kl: 0.837          
                                        

## Training Cluster model

In [ ]:
!svc train-cluster --output-path drive/MyDrive/aimodel/logs/44k/kmeans.pt

[07:48:38] INFO     [07:48:38] Version: 4.1.32                                        ]8;id=100209;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=399487;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py#31\31]8;;\
Usage: svc train-cluster [OPTIONS]
Try 'svc train-cluster -h' for help.

Error: Invalid value for '-i' / '--input-dir': Path 'dataset/44k' does not exist.


## Inference

In [ ]:
#@title Get the author's voice as a source
import random
NAME = str(random.randint(1, 49))
TYPE = "fsd50k" #@param ["", "digit", "dog", "fsd50k"]
CUSTOM_FILEPATH = "" #@param {type: "string"}
if CUSTOM_FILEPATH != "":
    NAME = CUSTOM_FILEPATH
else:
    # it is extremely difficult to find a voice that can download from the internet directly
    if TYPE == "dog":
        !wget -N f"https://huggingface.co/datasets/437aewuh/dog-dataset/resolve/main/dogs/dogs_{NAME:.0000}.wav" -O {NAME}.wav
    elif TYPE == "digit":
        # george, jackson, lucas, nicolas, ...
        !wget -N f"https://github.com/Jakobovski/free-spoken-digit-dataset/raw/master/recordings/0_george_{NAME}.wav" -O {NAME}.wav
    elif TYPE == "fsd50k":
        !wget -N f"https://huggingface.co/datasets/Fhrozen/FSD50k/blob/main/clips/dev/{10000+int(NAME)}.wav" -O {NAME}.wav
    else:
        !wget -N f"https://zunko.jp/sozai/utau/voice_{"kiritan" if NAME < 25 else "itako"}{NAME % 5 + 1}.wav" -O {NAME}.wav
from IPython.display import Audio, display
display(Audio(f"{NAME}.wav"))

for details.

fhttps://huggingface.co/datasets/Fhrozen/FSD50k/blob/main/clips/dev/10016.wav: Unsupported scheme ‘fhttps’.


In [1]:
#@title Use trained model
#@markdown **Put your .wav file in `so-vits-svc-fork/audio` directory**
from IPython.display import Audio, display
!svc infer drive/MyDrive/aimodel/original1.wav -m drive/MyDrive/aimodel/logs/44k/ -c drive/MyDrive/aimodel/logs/44k/config.json
display(Audio(f"drive/MyDrive/aimodel/original1.out.wav", autoplay=True))

/bin/bash: line 1: svc: command not found


ValueError: ignored

In [ ]:
from pydub import AudioSegment
import os

# Đường dẫn đến thư mục chứa file .wav đầu ra từ AI model
input_folder = 'drive/MyDrive/aimodel/output'
output_folder = 'drive/MyDrive/aimodel/adjusted_output'

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Duyệt qua mỗi file .wav trong thư mục input
for filename in os.listdir(input_folder):
    if filename.endswith('.wav'):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Đọc file âm thanh
        sound = AudioSegment.from_wav(input_path)

        # Tăng tốc độ phát lại lên 1.4 lần
        adjusted_sound = sound.speedup(playback_speed=1.4)

        # Lưu file đã điều chỉnh
        adjusted_sound.export(output_path, format='wav')


ModuleNotFoundError: ignored

In [ ]:
import os

# Đường dẫn đến thư mục chứa file .wav ban đầu và thư mục đầu ra
input_folder = 'drive/MyDrive/aimodel'
output_folder = 'drive/MyDrive/aimodel/output'
model_path = 'drive/MyDrive/aimodel/logs/44k/' # Đường dẫn đến model
config_path = 'drive/MyDrive/aimodel/logs/44k/config.json' # Đường dẫn đến file config

# Tạo thư mục output nếu chưa tồn tại
os.makedirs(output_folder, exist_ok=True)

# Duyệt qua mỗi file .wav trong thư mục input
for filename in os.listdir(input_folder):
    if filename.endswith('.wav'):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename.replace('.wav', '.out.wav'))

        # Chạy lệnh svc infer cho mỗi file
        !svc infer {input_path} -m {model_path} -c {config_path}


[08:12:36] INFO     [08:12:36] Since model_path is a directory, use                  ]8;id=774541;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=429793;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py#273\273]8;;\
                    drive/MyDrive/aimodel/logs/44k/G_62.pth                                         
Some weights of HubertModel were not initialized from the model checkpoint at lengyue233/content-vec-best and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[08:12:40] WARNING  [08:12:40]                                                       ]8;id=947306;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=2426;file:///usr/lib/python3.10/warnings.py#109\109]8;;

In [ ]:
!pip install pydub


In [ ]:
##@title Use trained model (with cluster)
!svc infer {NAME}.wav -s speaker -r 0.1 -m drive/MyDrive/so-vits-svc-fork/logs/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json -k drive/MyDrive/so-vits-svc-fork/logs/44k/kmeans.pt
display(Audio(f"{NAME}.out.wav", autoplay=True))

### Pretrained models

In [ ]:
#@title https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/tree/main
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/G_riri_220.pth"
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/config.json"

In [ ]:
!svc infer {NAME}.wav -c config.json -m G_riri_220.pth
display(Audio(f"{NAME}.out.wav", autoplay=True))

In [ ]:
#@title https://huggingface.co/therealvul/so-vits-svc-4.0/tree/main
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/G_166400.pth"
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/config.json"

In [ ]:
!svc infer {NAME}.wav --speaker "Pinkie {neutral}" -c config.json -m G_166400.pth
display(Audio(f"{NAME}.out.wav", autoplay=True))